In [20]:
import org.apache.spark.sql.SparkSession

lastException: Throwable = null


In [21]:
val spark = SparkSession.
    builder.
    config("spark.ui.port", "0").
    config("spark.sql.warehouse.dir", "/user/itv001389/warehouse").
    enableHiveSupport.
    master("yarn").
    appName("Getting Started - Spark with Scala").
    getOrCreate

spark = org.apache.spark.sql.SparkSession@4b325e55


org.apache.spark.sql.SparkSession@4b325e55

In [3]:
spark.sql("use raj")

[]

In [4]:
spark.sql("select count(*) from orders").show()

+--------+
|count(1)|
+--------+
|   68883|
+--------+



In [19]:
spark.sql("show tables").show(false)

+--------+------------------+-----------+
|database|tableName         |isTemporary|
+--------+------------------+-----------+
|raj     |order_immu        |false      |
|raj     |orderitem_nor     |false      |
|raj     |orderitem_orc     |false      |
|raj     |orderitem_par     |false      |
|raj     |order_items       |false      |
|raj     |order_items_part  |false      |
|raj     |order_oi          |false      |
|raj     |order_oi_cast     |false      |
|raj     |order_oi_cast1    |false      |
|raj     |order_oi_part     |false      |
|raj     |order_oi_part1    |false      |
|raj     |order_oi_part2    |false      |
|raj     |order_oi_partit   |false      |
|raj     |order_oi_partition|false      |
|raj     |order_oi_sum1     |false      |
|raj     |order_oi_sum2     |false      |
|raj     |order_orc         |false      |
|raj     |orders            |false      |
|raj     |orders_buck       |false      |
|raj     |orders_buck1      |false      |
+--------+------------------+-----

In [1]:
val df=spark.read.parquet("hdfs://m01.itversity.com:9000/user/itv001389/warehouse/raj.db/orders")

Waiting for a Spark session to start...

df = [order_id: int, Order_date: timestamp ... 2 more fields]


[order_id: int, Order_date: timestamp ... 2 more fields]

In [40]:
df.printSchema()

root
 |-- order_id: integer (nullable = true)
 |-- Order_date: timestamp (nullable = true)
 |-- Order_product_id: integer (nullable = true)
 |-- Order_status: string (nullable = true)



In [22]:
df.count()

68883

In [41]:
df.show(5,false)

+--------+-------------------+----------------+---------------+
|order_id|Order_date         |Order_product_id|Order_status   |
+--------+-------------------+----------------+---------------+
|1       |2013-07-25 00:00:00|11599           |CLOSED         |
|2       |2013-07-25 00:00:00|256             |PENDING_PAYMENT|
|3       |2013-07-25 00:00:00|12111           |COMPLETE       |
|4       |2013-07-25 00:00:00|8827            |CLOSED         |
|5       |2013-07-25 00:00:00|11318           |COMPLETE       |
+--------+-------------------+----------------+---------------+
only showing top 5 rows



In [46]:
val df1=spark.read.format("orc").load("hdfs://m01.itversity.com:9000/user/itv001389/warehouse/raj.db/order_orc")

df1 = [Order_id: int, Order_date: timestamp ... 2 more fields]


lastException: Throwable = null


[Order_id: int, Order_date: timestamp ... 2 more fields]

In [47]:
df1.printSchema()

root
 |-- Order_id: integer (nullable = true)
 |-- Order_date: timestamp (nullable = true)
 |-- Order_product_id: integer (nullable = true)
 |-- order_status: string (nullable = true)



In [48]:
df1.count()

68883

In [49]:
df1.show(5,false)

+--------+-------------------+----------------+---------------+
|Order_id|Order_date         |Order_product_id|order_status   |
+--------+-------------------+----------------+---------------+
|1       |2013-07-25 00:00:00|11599           |CLOSED         |
|2       |2013-07-25 00:00:00|256             |PENDING_PAYMENT|
|3       |2013-07-25 00:00:00|12111           |COMPLETE       |
|4       |2013-07-25 00:00:00|8827            |CLOSED         |
|5       |2013-07-25 00:00:00|11318           |COMPLETE       |
+--------+-------------------+----------------+---------------+
only showing top 5 rows



In [6]:
val df_table=spark.read.table("raj.orders")

df_table = [order_id: int, Order_date: timestamp ... 2 more fields]


[order_id: int, Order_date: timestamp ... 2 more fields]

####   saveAsTable will create a new table if the table doesn't exist and load the data.If table exists need to use overwrite
####   Append is used If we want to maintain the historical data.

In [14]:
df_table.write.format("orc").mode("append").saveAsTable("raj.orders_write1")

In [9]:
spark.sql("select * from raj.orders_write1 limit 5").show(false)

+--------+-------------------+----------------+---------------+
|order_id|Order_date         |Order_product_id|Order_status   |
+--------+-------------------+----------------+---------------+
|1       |2013-07-25 00:00:00|11599           |CLOSED         |
|2       |2013-07-25 00:00:00|256             |PENDING_PAYMENT|
|3       |2013-07-25 00:00:00|12111           |COMPLETE       |
|4       |2013-07-25 00:00:00|8827            |CLOSED         |
|5       |2013-07-25 00:00:00|11318           |COMPLETE       |
+--------+-------------------+----------------+---------------+



##### insertInto is used when a table is already exists and need to change file format 
#####  overwrite is used if we dont want to maintain the historical data

In [15]:
df_table.write.format("orc").mode("overwrite").insertInto("raj.orders_write")

In [16]:
spark.sql("select * from raj.orders_write").count()

68883

In [61]:
spark.read.table("raj.orders_write").show(5,false)

+--------+-------------------+----------------+---------------+
|order_id|Order_date         |Order_product_id|Order_status   |
+--------+-------------------+----------------+---------------+
|1       |2013-07-25 00:00:00|11599           |CLOSED         |
|2       |2013-07-25 00:00:00|256             |PENDING_PAYMENT|
|3       |2013-07-25 00:00:00|12111           |COMPLETE       |
|4       |2013-07-25 00:00:00|8827            |CLOSED         |
|5       |2013-07-25 00:00:00|11318           |COMPLETE       |
+--------+-------------------+----------------+---------------+
only showing top 5 rows



In [63]:
spark.sql("describe formatted raj.orders_write").show(false)

+----------------------------+--------------------------------------------------------------------------+-------+
|col_name                    |data_type                                                                 |comment|
+----------------------------+--------------------------------------------------------------------------+-------+
|order_id                    |int                                                                       |null   |
|Order_date                  |timestamp                                                                 |null   |
|Order_product_id            |int                                                                       |null   |
|Order_status                |string                                                                    |null   |
|                            |                                                                          |       |
|# Detailed Table Information|                                                          

In [7]:
!ls

01_Table_Creations.ipynb	      07_Bucketing_practice.ipynb
02_Order_table_creations.ipynb	      08_Handling_small_files.ipynb
03_OrderItem_table_creation.ipynb     09_Spark.ipynb
04_Order_OrderItem_Joins.ipynb	      filename.hql
05_Order_OrderItems_Partitions.ipynb  ${system:java.io.tmpdir}
06_Msck_Practise.ipynb		      test.txt


In [10]:
!hadoop fs -ls hdfs://m01.itversity.com:9000/user/itv001389/warehouse/raj.db/orders_write

Found 2 items
-rw-r--r--   3 itv001389 supergroup          0 2021-10-18 12:11 hdfs://m01.itversity.com:9000/user/itv001389/warehouse/raj.db/orders_write/_SUCCESS
-rw-r--r--   3 itv001389 supergroup     487493 2021-10-18 12:11 hdfs://m01.itversity.com:9000/user/itv001389/warehouse/raj.db/orders_write/part-00000-47d17d10-9b1c-4dc5-87de-6ee1efb948cf-c000.snappy.parquet


In [5]:
val df=spark.read.parquet("hdfs://m01.itversity.com:9000/user/itv001389/warehouse/raj.db/orders")

df = [order_id: int, Order_date: timestamp ... 2 more fields]


[order_id: int, Order_date: timestamp ... 2 more fields]

In [6]:
df.write.format("orc").mode("overwrite").save("hdfs://m01.itversity.com:9000/user/itv001389/warehouse/raj.db/sample")

In [ ]:
spark.sql("show tables").show(30,false)

In [7]:
df.createOrReplaceTempView("Orders_temp")

In [ ]:
spark.sql("select * from Orders_temp").show(false)

In [23]:
val df1=spark.read.parquet("hdfs://m01.itversity.com:9000/user/itv001389/warehouse/raj.db/order_items_par")

df1 = [order_item_id: int, order_item_order_id: int ... 4 more fields]


lastException: Throwable = null


[order_item_id: int, order_item_order_id: int ... 4 more fields]

In [24]:
df1.write.format("orc").mode("overwrite").save("hdfs://m01.itversity.com:9000/user/itv001389/warehouse/raj.db/order_items_orc_spark")

In [8]:
spark.sql("drop table Orders_temp")

[]

In [11]:
    spark.sql("create table raj.order_o (order_id int,order_date timestamp,order_product_id int,order_status string) row format delimited fields terminated by ',' stored as textfile location 'hdfs://m01.itversity.com:9000/user/itv001389/warehouse/raj.db/order.simple' ")

lastException: Throwable = null


[]

In [12]:
spark.sql("describe formatted raj.order_o").show(false)

+----------------------------+--------------------------------------------------------------------------+-------+
|col_name                    |data_type                                                                 |comment|
+----------------------------+--------------------------------------------------------------------------+-------+
|order_id                    |int                                                                       |null   |
|order_date                  |timestamp                                                                 |null   |
|order_product_id            |int                                                                       |null   |
|order_status                |string                                                                    |null   |
|                            |                                                                          |       |
|# Detailed Table Information|                                                          

In [13]:
spark.sql("insert into table raj.order_o select order_id,cast(order_date as date),order_product_id,order_status from raj.orders")

[]

In [33]:
spark.sql("describe formatted raj.order_o").show(false)

+----------------------------+--------------------------------------------------------------------------+-------+
|col_name                    |data_type                                                                 |comment|
+----------------------------+--------------------------------------------------------------------------+-------+
|order_id                    |int                                                                       |null   |
|order_date                  |timestamp                                                                 |null   |
|order_product_id            |int                                                                       |null   |
|order_status                |string                                                                    |null   |
|                            |                                                                          |       |
|# Detailed Table Information|                                                          

In [8]:
spark.sql("select * from raj.order_o limit 5 ").show(false)

lastException = null


org.apache.spark.sql.AnalysisException: Table or view not found: `raj`.`order_o`; line 1 pos 14;
'GlobalLimit 5
+- 'LocalLimit 5
   +- 'Project [*]
      +- 'UnresolvedRelation `raj`.`order_o`


In [4]:
spark.sql("drop table raj.order_o")

[]

In [10]:
val df=spark.read.parquet("hdfs://m01.itversity.com:9000/user/itv001389/warehouse/raj.db/orders")

df = [order_id: int, Order_date: timestamp ... 2 more fields]


lastException: Throwable = null


[order_id: int, Order_date: timestamp ... 2 more fields]

In [12]:
df.write.format("orc").mode("overwrite").save("hdfs://m01.itversity.com:9000/user/itv001389/warehouse/raj.db/orders_1")

lastException: Throwable = null


In [14]:
df.createOrReplaceTempView("test")

In [15]:
spark.sql("select * from test limit 5").show(false)

+--------+-------------------+----------------+---------------+
|order_id|Order_date         |Order_product_id|Order_status   |
+--------+-------------------+----------------+---------------+
|1       |2013-07-25 00:00:00|11599           |CLOSED         |
|2       |2013-07-25 00:00:00|256             |PENDING_PAYMENT|
|3       |2013-07-25 00:00:00|12111           |COMPLETE       |
|4       |2013-07-25 00:00:00|8827            |CLOSED         |
|5       |2013-07-25 00:00:00|11318           |COMPLETE       |
+--------+-------------------+----------------+---------------+



In [16]:
spark.sql("select order_id,cast(order_date as date),order_product_id,order_status from test")

[order_id: int, order_date: date ... 2 more fields]

In [17]:
val df1=spark.sql("select order_id,cast(order_date as date),order_product_id,order_status from test")

df1 = [order_id: int, order_date: date ... 2 more fields]


[order_id: int, order_date: date ... 2 more fields]

In [18]:
df1.show(5,false)

+--------+----------+----------------+---------------+
|order_id|order_date|order_product_id|order_status   |
+--------+----------+----------------+---------------+
|1       |2013-07-25|11599           |CLOSED         |
|2       |2013-07-25|256             |PENDING_PAYMENT|
|3       |2013-07-25|12111           |COMPLETE       |
|4       |2013-07-25|8827            |CLOSED         |
|5       |2013-07-25|11318           |COMPLETE       |
+--------+----------+----------------+---------------+
only showing top 5 rows



In [19]:
df1.write.mode("overwrite").saveAsTable("raj.order_1")

java.util.NoSuchElementException: None.get